In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
import nltk
import re
import PyPDF2
from gensim.utils import simple_preprocess
from numpy.linalg import norm

In [2]:
# Load the Model Dataset
model_df = pd.read_csv('nyc_jobs_1.csv')

In [3]:
model_df

,job_id,agency,posting_type,of_positions,business_title,civil_service_title,title_code_no,level,job_category,full_time_part_time_indicator,...,additional_information,to_apply,hours_shift,work_location_1,recruitment_contact,residency_requirement,posting_date,post_until,posting_updated,process_date
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,NaN,NaN,...,"Salary range for this position is: $42,405 - $...",NaN,NaN,NaN,NaN,New York City residency is generally required ...,2011-06-24T00:00:00,NaN,2011-06-24T00:00:00,2018-07-17T00:00:00
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,NaN,F,...,NaN,"In addition to applying through this website, ...",NaN,NaN,NaN,New York City residency is generally required ...,2012-01-26T00:00:00,NaN,2012-01-26T00:00:00,2018-07-17T00:00:00
2,102221,DEPT OF ENVIRONMENT PROTECTION,External,1,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,20616,0,NaN,F,...,Appointments are subject to OMB approval,click the apply now button,35 hours per week/day,NaN,NaN,New York City Residency is not required for th...,2012-06-21T00:00:00,NaN,2012-09-07T00:00:00,2018-07-17T00:00:00
3,102221,DEPT OF ENVIRONMENT PROTECTION,Internal,1,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,20616,0,NaN,F,...,Appointments are subject to OMB approval,click the apply now button,35 hours per week/day,NaN,NaN,New York City Residency is not required for th...,2012-06-21T00:00:00,NaN,2012-09-07T00:00:00,2018-07-17T00:00:00
4,114352,DEPT OF ENVIRONMENT PROTECTION,Internal,5,Deputy Plant Chief,SENIOR STATIONARY ENGINEER (EL,91639,0,NaN,F,...,Appointments are subject to OMB approval Fo...,"Click ""Apply Now"" button",40 per week / day,Various,NaN,New York City residency is generally required ...,2012-12-12T00:00:00,NaN,2012-12-13T00:00:00,2018-07-17T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,354163,HOUSING PRESERVATION & DVLPMNT,Internal,1,"Project Manager, Neighborhood Stabilization",COMMUNITY COORDINATOR,56058,0,"Engineering, Architecture, & Planning",F,...,*** PLEASE NOTE: THE ACTUAL SALARY FOR THIS PO...,APPLY ONLINE,NaN,100 Gold Street,NaN,New York City residency is generally required ...,2018-07-16T00:00:00,2018-08-15T00:00:00,2018-07-16T00:00:00,2018-07-17T00:00:00
3416,354164,OFFICE OF MANAGEMENT & BUDGET,Internal,1,ANALYST,INTERGOVERNMENTAL,BUDGET ANALYST (OMB),6088,1,Communications & Intergovernmental Affairs Pol...,...,QUALIFICATIONS: â€¢\tMust possess strong comp...,"Assistant Analyst ($43,618+): Bachelor's degre...","For City employees, please go to Employee Self...",NaN,255 Greenwich Street,NaN,New York City residency is generally required ...,2018-07-16T00:00:00,2018-07-16T00:00:00,2018-07-17T00:00:00
3417,354164,OFFICE OF MANAGEMENT & BUDGET,External,1,ANALYST,INTERGOVERNMENTAL,BUDGET ANALYST (OMB),6088,1,Communications & Intergovernmental Affairs Pol...,...,QUALIFICATIONS: â€¢\tMust possess strong comp...,"Assistant Analyst ($43,618+): Bachelor's degre...","For City employees, please go to Employee Self...",NaN,255 Greenwich Street,NaN,New York City residency is generally required ...,2018-07-16T00:00:00,2018-07-16T00:00:00,2018-07-17T00:00:00
3418,354182,DEPT OF HEALTH/MENTAL HYGIENE,External,1,"Deputy Director of Operations, Office of the E...",CITY RESEARCH SCIENTIST,21744,3,"Health Policy, Research & Analysis Social Serv...",F,...,NaN,Apply online with a cover letter to https://a1...,NaN,NaN,NaN,New York City residency is generally required ...,2018-07-16T00:00:00,2018-07-27T00:00:00,2018-07-16T00:00:00,2018-07-17T00:00:00


In [4]:
model_df.columns

Index(['job_id', 'agency', 'posting_type', 'of_positions', 'business_title',
       'civil_service_title', 'title_code_no', 'level', 'job_category',
       'full_time_part_time_indicator', 'salary_range_from', 'salary_range_to',
       'salary_frequency', 'work_location', 'division_work_unit',
       'job_description', 'minimum_qual_requirements', 'preferred_skills',
       'additional_information', 'to_apply', 'hours_shift', 'work_location_1',
       'recruitment_contact', 'residency_requirement', 'posting_date',
       'post_until', 'posting_updated', 'process_date'],
      dtype='object')

In [5]:
model_df = model_df[['business_title', 'job_description', 'minimum_qual_requirements', 'preferred_skills']]
model_df.head()

,business_title,job_description,minimum_qual_requirements,preferred_skills
0,Account Manager,Division of Economic & Financial Opportunity (...,1.\tA baccalaureate degree from an accredited ...,â€¢\tExcellent interpersonal and organizationa...
1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",The New York City Department of Small Business...,1. A baccalaureate degree from an accredited c...,NaN
2,Project Specialist,"Under direct supervision, perform elementary e...",A Baccalaureate degree from an accredited coll...,NaN
3,Project Specialist,"Under direct supervision, perform elementary e...",A Baccalaureate degree from an accredited coll...,NaN
4,Deputy Plant Chief,"Under general direction, is in responsible cha...",1. Six years of full-time satisfactory experie...,NaN


In [6]:
model_df['data'] = model_df.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
model_df.drop(['business_title', 'job_description', 'minimum_qual_requirements', 'preferred_skills'], axis=1, inplace=True)
print(model_df.head())

                                                data
0  Account Manager Division of Economic & Financi...
1  EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT The N...
2  Project Specialist Under direct supervision, p...
3  Project Specialist Under direct supervision, p...
4  Deputy Plant Chief Under general direction, is...


C:\Users\Mangukiya Ansh\AppData\Local\Temp\ipykernel_29688\2248679414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['data'] = model_df.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
C:\Users\Mangukiya Ansh\AppData\Local\Temp\ipykernel_29688\2248679414.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.drop(['business_title', 'job_description', 'minimum_qual_requirements', 'preferred_skills'], axis=1, inplace=True)


In [7]:
data = list(model_df['data'])
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

In [8]:
tagged_data[10]

TaggedDocument(words=['network', 'engineer', 'doitt', 'provides', 'for', 'the', 'sustained', ',', 'efficient', 'and', 'effective', 'delivery', 'of', 'it', 'services', ',', 'infrastructure', 'and', 'telecommunications', 'to', 'enhance', 'service', 'delivery', 'to', 'new', 'york', 'city', "'s", 'residents', ',', 'businesses', ',', 'employees', 'and', 'visitors', '.', 'as', 'the', 'city', "'s", 'technology', 'leader', ',', 'doitt', 'is', 'responsible', 'for', 'maintaining', 'the', 'foundational', 'it', 'infrastructure', 'and', 'systems', 'that', 'touch', 'every', 'aspect', 'of', 'city', 'life', 'from', 'public', 'safety', 'to', 'human', 'services', ',', 'from', 'education', 'to', 'economic', 'development', 'crossing', 'the', 'full', 'spectrum', 'of', 'governmental', 'operations', '.', 'the', 'successful', 'candidate', 'will', 'serve', 'as', 'a', 'network', 'engineer', 'reporting', 'to', 'it', 'services', 'division', '.', 'responsibilities', 'will', 'include', ':', 'â€¢', 'handle', 'all', 

In [9]:
model = Doc2Vec( window=7,
min_count = 5,
epochs = 100,
alpha = 0.001
)

In [12]:
model.build_vocab(tagged_data)


In [13]:
model.train(tagged_data,total_examples=model.corpus_count,epochs=model.epochs)

In [14]:
model.save('Resume_job_description.model')

In [15]:
# Load the Actual TNP Dataset

df = pd.read_csv('Preprocessed_SCET_TnP_Data.csv',usecols=[1,2,3])

In [16]:
df

,Company Name,Recruitment Type,Company Reqirements
0,Intellicon Pvt Ltd - Gandhinagar,On Campus\nThrough GTU ITAP Cell\n,Diploma/BE (EC) with good communication and co...
1,Just Green,On Campus,"Web development,Mobile application development..."
2,WhiteLion,On Campus,"Excellent knowledge of MS Office.,Highly motiv..."
3,D-mart,On Campus,BE Electrical with no active backlogs\r\n60% t...
4,National Group,On Campus,English must be good
...,...,...,...
512,ETP PLANT,On Campus,Be . chemical
513,HivePro,On Campus,"Problem-Solving and Critical Thinking,JAVA,Pyt..."
514,Xceed - A Unit of Safexpress Pvt Ltd,On Campus,"MySQL, MongoDB, SQLite, UI, Problem-Solving an..."
515,TCS - Cod vita,On Campus,"C, C++, C#, Java, Python, Pearl, Ruby, PHP"


In [17]:
df['Company  Reqirements']=df['Company  Reqirements'].str.lower()

In [18]:
df['Company Name'] = df['Company Name'].str.lower()

In [19]:
df[df.duplicated()]

,Company Name,Recruitment Type,Company Reqirements


In [20]:
df

,Company Name,Recruitment Type,Company Reqirements
0,intellicon pvt ltd - gandhinagar,On Campus\nThrough GTU ITAP Cell\n,diploma/be (ec) with good communication and co...
1,just green,On Campus,"web development,mobile application development..."
2,whitelion,On Campus,"excellent knowledge of ms office.,highly motiv..."
3,d-mart,On Campus,be electrical with no active backlogs\r\n60% t...
4,national group,On Campus,english must be good
...,...,...,...
512,etp plant,On Campus,be . chemical
513,hivepro,On Campus,"problem-solving and critical thinking,java,pyt..."
514,xceed - a unit of safexpress pvt ltd,On Campus,"mysql, mongodb, sqlite, ui, problem-solving an..."
515,tcs - cod vita,On Campus,"c, c++, c#, java, python, pearl, ruby, php"


In [21]:
df['Company  Reqirements'][0]

'diploma/be (ec) with good communication and computer skills,ccountable for stock planning and stock forecast'

In [22]:
simple_preprocess(df['Company  Reqirements'][0])

['diploma',
 'be',
 'ec',
 'with',
 'good',
 'communication',
 'and',
 'computer',
 'skills',
 'ccountable',
 'for',
 'stock',
 'planning',
 'and',
 'stock',
 'forecast']

In [23]:
def preprocessing(text):
    return simple_preprocess(text)

In [24]:
df['Tokenize_Requirements']=df['Company  Reqirements'].apply(preprocessing)

In [25]:
df

,Company Name,Recruitment Type,Company Reqirements,Tokenize_Requirements
0,intellicon pvt ltd - gandhinagar,On Campus\nThrough GTU ITAP Cell\n,diploma/be (ec) with good communication and co...,"[diploma, be, ec, with, good, communication, a..."
1,just green,On Campus,"web development,mobile application development...","[web, development, mobile, application, develo..."
2,whitelion,On Campus,"excellent knowledge of ms office.,highly motiv...","[excellent, knowledge, of, ms, office, highly,..."
3,d-mart,On Campus,be electrical with no active backlogs\r\n60% t...,"[be, electrical, with, no, active, backlogs, t..."
4,national group,On Campus,english must be good,"[english, must, be, good]"
...,...,...,...,...
512,etp plant,On Campus,be . chemical,"[be, chemical]"
513,hivepro,On Campus,"problem-solving and critical thinking,java,pyt...","[problem, solving, and, critical, thinking, ja..."
514,xceed - a unit of safexpress pvt ltd,On Campus,"mysql, mongodb, sqlite, ui, problem-solving an...","[mysql, mongodb, sqlite, ui, problem, solving,..."
515,tcs - cod vita,On Campus,"c, c++, c#, java, python, pearl, ruby, php","[java, python, pearl, ruby, php]"


In [26]:
from pdfminer.high_level import extract_text


In [27]:
resume = (extract_text('Priyanshi_Dobariya_1_20240823_135608_0000.pdf'))

In [28]:
print(resume)

Priyanshi Dobariya

Surat, Gujrat | P:+91 7984790671 | priyanshidobariya72@gmail.com
LinkedIn: linkedin.com/in/priyanshi-dobariya-974a58221 | GitHub: github.com/Priyanshidob

A B O U T   M E

I am an enthusiastic data professional deeply committed to my craft. I thrive in challenging and dynamic
environments, continually seeking fresh opportunities to expand my skill set. My area of expertise lies in
data analysis, and I am a self-motivated individual with a strong drive to learn and grow. I am excited to
bring my talents to new endeavors and take on new challenges.

L A N G U A G E S

English
Hindi
Gujarati

T E C H N I C A L   S K I L L S

Programming Languages: C, Java, Python, HTML, CSS, JavaScript, PHP
Technologies: Machine Learning, Data Science, Basic NLP, Flask, MongoDB, MySQL
Libraries/Frameworks: ReactJS, Matplotlib, Numpy, Pandas, Scikit-learn, Keras, TensorFlow

E D U C A T I O N  
B a c h e l o r   o f   E n g i n e e r i n g   a n d   T e c h n o l o g y                  

In [29]:

# pobj: Prepositional object
# dobj: Direct object
# nsubj: Nominal subject
# compound: Compound noun (usually technical terms or proper names)

import re
import spacy


nlp = spacy.load('en_core_web_sm')



def extract_dynamic_skills(text):
    doc = nlp(text)

    extracted_skills = set()

    for chunk in doc.noun_chunks:
        if chunk.root.dep_ in ('pobj', 'dobj', 'nsubj', 'compound'):
            extracted_skills.add(chunk.text)
    
    filtered_skills = set()
    for token in doc:
        if token.pos_ in ('PROPN', 'NOUN') and len(token.text) > 1:
            filtered_skills.add(token.text)
    
    return extracted_skills.union(filtered_skills)

In [30]:
# Demo

doc = nlp(resume)
for chunk in doc.noun_chunks:
    print(chunk.text,'-->',chunk.root.dep_)

Priyanshi Dobariya --> ROOT
Surat --> appos
LinkedIn --> nsubj
| GitHub --> ROOT
A B O --> appos
T --> appos
M E --> ROOT
I --> nsubj
an enthusiastic data --> attr
professional --> nsubj
my craft --> pobj
I --> nsubj
challenging and dynamic
environments --> pobj
fresh opportunities --> dobj
My area --> nsubj
expertise --> pobj
data analysis --> pobj
I --> nsubj
a self-motivated individual --> attr
a strong drive --> pobj
I --> nsubj
my talents --> dobj
new endeavors --> pobj
new challenges --> dobj
L A --> ROOT
N G U --> ROOT
A G E S --> appos
English --> ROOT
Hindi --> ROOT
Gujarati --> ROOT
T E C H --> ROOT
N I --> ROOT
C A --> ROOT
L --> ROOT
S K --> ROOT
I L L S --> ROOT
Programming Languages --> ROOT
C --> ROOT
Java --> conj
Python --> conj
HTML --> conj
CSS --> conj
JavaScript --> conj
PHP --> conj
Technologies --> ROOT
Machine Learning --> appos
Data Science --> conj
Basic NLP --> conj
Flask --> conj
MongoDB --> conj
Libraries/Frameworks --> ROOT
ReactJS --> appos
Matplotlib -->

In [31]:
skills = extract_dynamic_skills(resume)


In [32]:
temp = simple_preprocess(str(skills))

In [33]:
temp

['dataset',
 'recall',
 'deep',
 'learning',
 'techniques',
 'prediction',
 'scikit',
 'set',
 'college',
 'dropout',
 'neural',
 'python',
 'and',
 'machine',
 'learning',
 'libraries',
 'dogs',
 'model',
 'performance',
 'matplotlib',
 'understanding',
 'php',
 'various',
 'metrics',
 'cnn',
 'model',
 'ai',
 'dobariya',
 'tensorflow',
 'detection',
 'science',
 'surat',
 'programming',
 'cats',
 'basic',
 'developer',
 'flask',
 'overfitting',
 'github',
 'com',
 'priyanshidob',
 'convolutional',
 'nlp',
 'dog',
 'disease',
 'image',
 'the',
 'field',
 'plant',
 'dog',
 'machine',
 'craft',
 'english',
 'strong',
 'drive',
 'my',
 'area',
 'image',
 'classification',
 'image',
 'validation',
 'classification',
 'java',
 'metrics',
 'endeavors',
 'challenging',
 'and',
 'dynamic',
 'nenvironments',
 'prevent',
 'new',
 'challenges',
 'expertise',
 'technology',
 'hindi',
 'prize',
 'area',
 'numpy',
 'css',
 'keras',
 'talents',
 'machine',
 'fresh',
 'opportunities',
 'cgpa',
 'till

In [34]:
model = Doc2Vec.load('Resume_job_description.model')


In [35]:

# Convert the list to a string
final_skills = ' '.join(temp)


In [36]:
evalue = []

In [37]:
v1 = model.infer_vector(final_skills.split())
for i in range(df.shape[0]):
    v2 = model.infer_vector(df['Company  Reqirements'][i].split())
    similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
    evalue.append(similarity)

In [38]:
df['evalue'] = evalue

In [39]:
result = pd.DataFrame()

In [40]:
result['recommended_companies'] = df.sort_values(by=['evalue'],ascending=False)['Company Name'].head(10)

In [41]:
result['recuritment_type'] = df.sort_values(by=['evalue'],ascending=False)['Recruitment Type'].head(10)

In [42]:
result

,recommended_companies,recuritment_type
43,greeks lab solution,On Campus
57,ebizz infotech,On campus
409,axelor technologies,On Campus
120,agilecube technologies pvt ltd,On Campus
372,salesforce consultant,On Campus
65,mvix,On Campus
71,shell\n(through placement agency),Through Placement Agency-
139,rt camp,On Campus
171,webiots,On Campus
503,bigscal technologies pvt. ltd,On Campus


In [47]:
result.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>recommended_companies</th>\n      <th>recuritment_type</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>43</th>\n      <td>greeks lab solution</td>\n      <td>On Campus</td>\n    </tr>\n    <tr>\n      <th>57</th>\n      <td>ebizz infotech</td>\n      <td>On campus</td>\n    </tr>\n    <tr>\n      <th>409</th>\n      <td>axelor technologies</td>\n      <td>On Campus</td>\n    </tr>\n    <tr>\n      <th>120</th>\n      <td>agilecube technologies pvt ltd</td>\n      <td>On Campus</td>\n    </tr>\n    <tr>\n      <th>372</th>\n      <td>salesforce consultant</td>\n      <td>On Campus</td>\n    </tr>\n    <tr>\n      <th>65</th>\n      <td>mvix</td>\n      <td>On Campus</td>\n    </tr>\n    <tr>\n      <th>71</th>\n      <td>shell\\n(through placement agency)</td>\n      <td>Through Placement Agency-</td>\n    </tr>\n    <tr>\n      <th>139</th>\n      <td>rt camp</t

In [46]:
import pickle

pickle.dump(model,open('Recommendation_model.pkl','wb'))